# Works with python 3.6

In [1]:
import sys
import os
import numpy as np
sys.path.insert(0, '..')

In [2]:
import cv2

In [3]:
from gait_analysis.Datasets import TumGAID_Dataset
from gait_analysis import settings as S

In [4]:
import numpy as np
import torch
import torchvision as torchvision
import torchvision.transforms as transforms
import os
import timeit
import scipy.io
import numpy as np
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt


tumgaid_default_args = {
    'load_pose': True,
    'load_pose_options': {
        'D': 2,
        'body_keypoints_include_list': ['LAnkle',
                                        'RAnkle',
                                        'LKnee',
                                        'RKnee',
                                        'RHip',
                                        'LHip']
    },
    'load_flow': True,
    'load_flow_options': {
        'method': 'dense',
        'load_patches': True,
        'load_patch_options': {
            'patch_size': 2
        }
    },
    'load_scene': True,
    'load_scene_options': {
        'grayscale': False,
        'load_tracked': False
    },
    'include_scenes': ['b01', 'b02', 'n01', 'n02', 's01', 's02'],
}
dset = TumGAID_Dataset(S.tumgaid_root,
                       S.tumgaid_preprocessing_root,
                       S.tumgaid_annotations_root,
                       tumgaid_default_args)
#inputs, annotations = dset[0]
dset.p_nums

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 301, 302, 303, 304, 305]

# We need to write a wrapper class to get rid of the pandas datatype

In [5]:
class DsetWrapper():
    _dset = None
    def __init__(self, dset):
        self._dset = dset
    
    def __len__(self):
        return len(self._dset)
    
    def __getitem__(self, idx):
        data, annotations = self._dset[idx]
        targets = self.annotations_to_targets(annotations)
        return data, targets
        
    def annotations_to_targets(self, annotations):
        '''
        IN_THE_AIR == TRUE
        '''
        left_foot = annotations.left_foot.values == 'IN_THE_AIR'    
        right_foot = annotations.right_foot.values == 'IN_THE_AIR'
        frame_id = annotations.frame_id.values
        return 1.0 * np.stack([frame_id, left_foot, right_foot], axis=1)
        

dset = TumGAID_Dataset(S.tumgaid_root,
                       S.tumgaid_preprocessing_root,
                       S.tumgaid_annotations_root,
                       tumgaid_default_args)
dset_wrap = DsetWrapper(dset)
trainloader = DataLoader(dset_wrap, batch_size=1, shuffle=True)        
        
i = 0

for k in trainloader:
    i += 1
    data, targets = k
    print(targets.shape)
    print(data.keys())
    if i == 2:
        break
print(i)

/home/sandro/anaconda2/envs/gait_36/lib/python3.6/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


torch.Size([1, 63, 3])
dict_keys(['pose_keypoints', 'flow_maps', 'scene_images', 'total_images'])
torch.Size([1, 60, 3])
dict_keys(['pose_keypoints', 'flow_maps', 'scene_images', 'total_images'])
2


# Debugging code snippets

In [6]:
dset = TumGAID_Dataset(S.tumgaid_root,
                       S.tumgaid_preprocessing_root,
                       S.tumgaid_annotations_root,
                       tumgaid_default_args)
#inputs, annotations = dset[0]
print(dset.p_nums)
data, annotations = dset[0]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 301, 302, 303, 304, 305]


/home/sandro/anaconda2/envs/gait_36/lib/python3.6/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [7]:
data

{'pose_keypoints': [[array([245.852, 286.37 ]),
   array([244.542, 350.247]),
   array([184.517, 393.346]),
   array([266.713, 282.438]),
   array([287.624, 330.745]),
   array([294.137, 392.073])],
  [array([256.337, 287.679]),
   array([262.814, 350.272]),
   array([202.82 , 390.759]),
   array([275.888, 286.288]),
   array([292.88 , 334.602]),
   array([294.18 , 392.081])],
  [array([265.452, 285.012]),
   array([285.003, 341.191]),
   array([223.655, 382.895]),
   array([277.212, 279.827]),
   array([298.076, 332.019]),
   array([295.446, 392.035])],
  [array([266.76 , 277.222]),
   array([298.146, 332.029]),
   array([240.65 , 382.945]),
   array([282.406, 274.571]),
   array([296.775, 334.608]),
   array([296.708, 392.084])],
  [array([283.732, 278.511]),
   array([311.134, 338.547]),
   array([265.453, 381.564]),
   array([291.558, 275.892]),
   array([298.026, 337.221]),
   array([296.777, 392.083])],
  [array([295.479, 277.199]),
   array([324.193, 337.245]),
   array([286.343

In [27]:
def annotations_to_targets(annotations):
    '''
    IN_THE_AIR == TRUE
    '''
    left_foot = annotations.left_foot.values == 'IN_THE_AIR'    
    right_foot = annotations.right_foot.values == 'IN_THE_AIR'
    frame_id = annotations.frame_id.values
    return 1.0 * np.stack([frame_id, left_foot, right_foot], axis=1)

annotations_to_targets(annotations)

array([[ 0.,  0.,  0.],
       [ 1.,  0.,  1.],
       [ 2.,  0.,  1.],
       [ 3.,  0.,  1.],
       [ 4.,  0.,  1.],
       [ 5.,  0.,  1.],
       [ 6.,  0.,  1.],
       [ 7.,  0.,  1.],
       [ 8.,  0.,  1.],
       [ 9.,  0.,  1.],
       [10.,  0.,  1.],
       [11.,  0.,  1.],
       [12.,  0.,  0.],
       [13.,  0.,  0.],
       [14.,  0.,  0.],
       [15.,  0.,  0.],
       [16.,  1.,  0.],
       [17.,  1.,  0.],
       [18.,  1.,  0.],
       [19.,  1.,  0.],
       [20.,  1.,  0.],
       [21.,  1.,  0.],
       [22.,  1.,  0.],
       [23.,  1.,  0.],
       [24.,  1.,  0.],
       [25.,  1.,  0.],
       [26.,  1.,  0.],
       [27.,  1.,  0.],
       [28.,  0.,  0.],
       [29.,  0.,  0.],
       [30.,  0.,  0.],
       [31.,  0.,  1.],
       [32.,  0.,  1.],
       [33.,  0.,  1.],
       [34.,  0.,  1.],
       [35.,  0.,  1.],
       [36.,  0.,  1.],
       [37.,  0.,  1.],
       [38.,  0.,  1.]])